# Batch Normalization

## Introdução

Batch Normalization é uma técnica que permite normalizar automaticamente os valores que atravessam uma camada da rede neural. Normalmente o uso do BatchNorm permite um treinamento mais rápido e com menores cuidados na inicialização dos pesos. Permite que redes profundas sejam treinadas mais facilmente. 

A normalização é feita para que o resultado tenha média zero e variância unitária, porém, em seguida, o resultado é
escalado pelo fator $\gamma$ e somado ao fator $\beta$. Estes dois fatores são parâmetros que serão também otimizados durante o treinamento do gradiente descendente.

Atualmente a técnica de batch normalization é utilizada em todas as redes profundas. 

A camada de batch normalization é colocada entre a camada densa ou convolucional e antes da camada de ativação.

**Referência:**
- Ioffe, S. and Szegedy, C. (2015), Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift., in Francis R. Bach & David M. Blei, ed., 'ICML' , JMLR.org, , pp. 448-456. 
    [PDF:arxiv](http://arxiv.org/pdf/1502.03167.pdf) 

<img src='../figures/batchnorm_neuronios.png' width="900"></img>

<img src='../figures/batchnorm_equations.png' width="500"></a>

## Diferença entre fase de treinamento e fase de predição ou avaliação

A normalização ocorre de forma distinta na fase de treinamento e na fase de avaliação:
- em treinamento: a média e variância ($\mu$ e $\sigma^2$) são estimados a partir dos valores
das amostras no mini-batch:
    - `running_mean`
    - `running_var`
- em avaliação: a média e variância é calculada pela média móvel, definida pelo parâmetro momentum ($\lambda$):

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{\mu_t} + (1 - \lambda) \boldsymbol{m}_{t-1} &&
\boldsymbol{v}_{t} =   \lambda \boldsymbol{\sigma^{2}_t} + (1 - \lambda) \boldsymbol{v}_{t-1} 
\end{align*}

## Importações

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display

import sys, copy
import numpy as np
import numpy.random as nr
import pandas as pd

import torch

np.set_printoptions(precision=2, suppress=True)
nr.seed(23456)


## nn.BatchNorm1d - Entrada com dimensão 2

- Documentação oficial: http://pytorch.org/docs/master/nn.html#normalization-layers

Quando a entrada da camada tem duas dimensões (amostras e atributos) a normalização por atributo. Ou seja, calcula-se a estatística (média e variância) para cada coluna da matriz de dados, em cada *mini-batch*.

## Criação rede com apenas uma camada BatchNorm

In [18]:
torch.manual_seed(1234)

momentum = 0.1
model = torch.nn.BatchNorm1d(5, momentum=momentum)        # Rede formada de apenas uma camada BatchNorm

## Parâmetros iniciais, mv_mean = 0. e mv_var = 1.

In [19]:
print('Training:', model.training)
gamma, beta, mv_mean, mv_var, _ = copy.deepcopy(model.state_dict()).values()
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: True
weight
[0.03 0.4  0.26 0.37 0.06]
bias
[0. 0. 0. 0. 0.]
running_mean
[0. 0. 0. 0. 0.]
running_var
[1. 1. 1. 1. 1.]
num_batches_tracked
0


## Entrada x (não normalizada)

In [20]:
x = torch.randn(4, 5) * 100. # Entrada x
print('x:\n',x)

x:
 tensor([[  66.8374,   -5.9658,  -46.7498,  -31.4352,  -71.4080],
        [-108.3124,  -55.4724,   97.1685,  -51.5009,  142.5527],
        [  79.8685, -149.4876,  147.7784,  -16.9624,  -99.1857],
        [-145.6908,   25.6292,  -40.3047,   41.9527,   93.8027]])


## Forward em treinamento: faz predict usando média e var do mini-batch e atualiza mv_mean e mv_var

In [21]:
print('Training:', model.training)
y = model(x)                           # Forward

Training: True


## Parâmetros: running_mean e running_var estimados com momentum 10% atual e 90% do anterior

In [22]:
print('Training:', model.training)
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: True
weight
[0.03 0.4  0.26 0.37 0.06]
bias
[0. 0. 0. 0. 0.]
running_mean
[-2.68 -4.63  3.95 -1.45  1.64]
running_var
[1365.08  585.32  962.82  162.53 1433.43]
num_batches_tracked
1


## Conferindo os novos valores de running_mean e running_var

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{\mu_t} + (1 - \lambda) \boldsymbol{m}_{t-1} &&
\boldsymbol{v}_{t} =   \lambda \boldsymbol{\sigma^{2}_t} + (1 - \lambda) \boldsymbol{v}_{t-1} 
\end{align*}

In [23]:
print('old mv_mean, mv_var:', mv_mean.numpy(), mv_var.numpy())

mv_mean_new = momentum * x.data.mean(0) + (1 - momentum) * mv_mean
mv_var_new  = momentum * x.data.var(0,unbiased=True) + (1 - momentum) * mv_var

print('my new running mean:',mv_mean_new.numpy())
print('my new running var :',mv_var_new.numpy())

old mv_mean, mv_var: [0. 0. 0. 0. 0.] [1. 1. 1. 1. 1.]
my new running mean: [-2.68 -4.63  3.95 -1.45  1.64]
my new running var : [1365.08  585.32  962.82  162.53 1433.43]


## Saída  y

In [24]:
y

tensor([[ 0.0268,  0.2450, -0.2638, -0.1785, -0.0494],
        [-0.0233, -0.0555,  0.1765, -0.3898,  0.0709],
        [ 0.0306, -0.6263,  0.3313, -0.0261, -0.0650],
        [-0.0341,  0.4368, -0.2441,  0.5943,  0.0435]],
       grad_fn=<ThnnBatchNormBackward>)

## Conferindo com código próprio

Em treinamento, a normalização é feita com a estatística do mini-batch

$$  y = \frac{x - mean[x]}{ \sqrt{Var[x] + \epsilon}} * gamma + beta $$

In [25]:
mean = x.mean(dim=0).data
var  = x.var(dim=0, unbiased=False).data # Veja que aqui é unbiased
print('mean:',mean.numpy())
print('var: ',var.numpy())
y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)

mean: [-26.82 -46.32  39.47 -14.49  16.44]
var:  [10231.36  4383.12  7214.42  1212.21 10743.99]
tensor([[ 0.0268,  0.2450, -0.2638, -0.1785, -0.0494],
        [-0.0233, -0.0555,  0.1765, -0.3898,  0.0709],
        [ 0.0306, -0.6263,  0.3313, -0.0261, -0.0650],
        [-0.0341,  0.4368, -0.2441,  0.5943,  0.0435]])


## Modo predict sem treinamento

In [26]:
model.training = False
y = model(x)                           # Forward

Observe que agora, o running_mean e running_var serão utilizados

In [27]:
print('Training:', model.training)
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: False
weight
[0.03 0.4  0.26 0.37 0.06]
bias
[0. 0. 0. 0. 0.]
running_mean
[-2.68 -4.63  3.95 -1.45  1.64]
running_var
[1365.08  585.32  962.82  162.53 1433.43]
num_batches_tracked
1


## Resultado da rede no modo eval(), note que os valores não ficaram tão normalizados (por quê?)

In [28]:
y

tensor([[ 0.0545, -0.0221, -0.4245, -0.8624, -0.1125],
        [-0.0829, -0.8446,  0.7806, -1.4395,  0.2170],
        [ 0.0647, -2.4063,  1.2045, -0.4462, -0.1553],
        [-0.1122,  0.5027, -0.3706,  1.2482,  0.1419]],
       grad_fn=<ThnnBatchNormBackward>)

## Conferindo com código próprio, note que usa-se o running_mean e running_var

In [29]:
gamma, beta, mv_mean, mv_var, _ = model.state_dict().values()

mean = mv_mean
var  = mv_var

y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)

tensor([[ 0.0545, -0.0221, -0.4245, -0.8624, -0.1125],
        [-0.0829, -0.8446,  0.7806, -1.4395,  0.2170],
        [ 0.0647, -2.4063,  1.2045, -0.4462, -0.1553],
        [-0.1122,  0.5027, -0.3706,  1.2482,  0.1419]])


## Exercícios

1. Coloque a rede no modo treinamento em um laço, de modo que a running_mean e running_var se aproximem da média e variância do mini-batch. Quantas execuções da rede serão necessárias?

In [14]:
model.training = True
#for i in range(100):
    #y = model(x)
    # Busque aqui os valores da média móvel da média e variância que estão sendo aprendidos
    #
    #
    #print(mv_mean.numpy(), mv_var.numpy())